<a href="https://colab.research.google.com/github/msbonus/swhp/blob/master/HP_CHP_PtH_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install iapws

In [0]:
!pip install iapws

all imports

In [0]:
import numpy as np
import copy
from enum import Enum
from iapws import iapws97

SI units

In [0]:
class SI:
  SIpfx = {
      'G': 10**9,
      'M': 10**6,
      'k': 10**3,
      'h': 10**2,
      'da': 10**1,
      '1': 1,
      'd': 10**-1,
      'c': 10**-2,
      'm': 10**-3,
      'u': 10**-6,
    }
  
  def prefix(pfx):
    return SI.SIpfx.get(pfx)
  
  def fmt(value):
    for pfx in SI.SIpfx:
      sipow = SI.SIpfx.get(pfx)
      if (np.log10(sipow) % 3 == 0):
        if (pfx == "1"): pfx = ""
        if abs(value) >= sipow: return "%g %s" % (value / sipow, pfx)

In [0]:
class CIndent:
  elem = "  "
  
  def __init__(self, ind = ""):
    self.ind = ind
    
  def __str__(self):
    return self.ind
    
  def __add__(self, other):
    return self.ind + other
    
  def inc(self):
    self.ind += CIndent.elem
    
  def dec(self):
    self.ind = self.ind[0:-len(CIndent.elem)]

  def reset(self):
    self.ind = ""
    
cind = CIndent()

In [0]:
def isNone(*vars, strict = False):
  if strict:
    for var in vars:
      if not (var is None): return False
    return True
  else:
    for var in vars:
      if (var is None): return True
    return False
  
def dropNone(*vars):
  ret = list()
  for var in vars:
    if not (var is None): ret.append(var)
  return ret

Water properties

In [0]:
pa = 101325/10**6 #MPa
TK0 = 273.15

def WaterCp(T):
  return (iapws97._Region1(T+TK0, pa))["cp"] * 1000 #J/(kg*K)

def WaterDensity(T):
  return 1/iapws97._Region1(T+TK0, pa)["v"] #kg/m3

Definition of Class "Heat-transfer fluid"

In [0]:
class HTF:
  def __init__(self, temperature = None, volflow = None, type = 'water'):
    self.temperature = temperature
    self.volflow = volflow
    self.type = type.lower()
    
  def __str__(self):
    return '%s at %g °C, %g m3/s' % (self.type, self.temperature, self.volflow)
  
  def __eq__(self, other):
    return ((self.type == other.type) and
           (self.temperature == other.temperature) and
           (self.volflow == other.volflow))
  
  def __gt__(self, other):
    return ((self.type == other.type) and
           (not isNone(self.temperature, other.temperature) and (self.temperature > other.temperature)) or
           (not isNone(self.volflow, other.volflow) and (self.volflow > other.volflow)))
  
  def __lt__(self, other):
    return ((self.type == other.type) and
           (not isNone(self.temperature, other.temperature) and (self.temperature < other.temperature)) or
           (not isNone(self.volflow, other.volflow) and (self.volflow < other.volflow)))
  
  def __add__(self, other):
    if isNone(self.temperature, self.volflow, strict = True):
      return HTF(other.temperature, other.volflow)
    elif isNone(other.temperature, other.volflow, strict = True):
      return HTF(self.temperature, self.volflow)
    else:
      print(cind + "Mixing " + str(self) + " with " + str(other), end="")
      temperature = (self.temperature * self.volflow +
                        other.temperature * other.volflow) / (self.volflow + other.volflow)
      volflow = self.volflow + other.volflow
      htfout = HTF(temperature, volflow)
      print(" -> " + str(htfout))
      return htfout
  
  def __sub__(self, other):
    if (self.temperature != other.temperature):
      raise ValueError("temperatures should match to subtract the flows")
    return HTF(self.temperature, self.volflow - other.volflow)
  
  def min(self, other):
    return HTF(min(dropNone(self.temperature, other.temperature)), min(dropNone(self.volflow, other.volflow)))
  
  def max(self, other):
    return HTF(max(dropNone(self.temperature, other.temperature)), max(dropNone(self.volflow, other.volflow)))
  
  def getCp(self):
    if self.type == 'water':
      return(WaterCp(self.temperature))
    else:
      return 0
    
  def getDens(self):
    if self.type == 'water':
      return(WaterDensity(self.temperature))
    else:
      return 0

Definition of Class "Efficiency"

In [0]:
class Efficiency:
  def __init__(self, elecEff = 0, fuelEff = 0):
    self.elecEff = elecEff
    self.fuelEff = fuelEff

Definition of power classes

In [0]:
class PowerType:
  def __init__(self, value = 0, SIprefix = '1'):
    self.value = value * SI.prefix(SIprefix)
    
  def __add__(self, other):
    return type(self)(self.value + other.value)

  def __sub__(self, other):
    return type(self)(self.value - other.value)
  
  def __mul__(self, other):
    return type(self)(self.value * other)
  
  def __str__(self):
    return "%s %sW" % (type(self).__name__, SI.fmt(self.value))

class ElecPower(PowerType):
  pass

class FuelPower(PowerType):
  pass

class Power:
  def __init__(self, *pwr_values):
    self.elecPower = ElecPower()
    self.fuelPower = FuelPower()
    for pwr in pwr_values:
      if type(pwr) is ElecPower:
        self.elecPower += pwr
      elif type(pwr) is FuelPower:
        self.fuelPower += pwr

  def all(self):
    return (self.elecPower.value + self.fuelPower.value)

  # Multiplying Power by efficiency in forward task 
  def __mul__(self, other):
    if type(other) is Efficiency:
      return self.elecPower.value * other.elecEff + self.fuelPower.value * other.fuelEff
    else:
      return 0
    
  def __add__(self, other):
    return Power(self.elecPower + other.elecPower, self.fuelPower + other.fuelPower)
  
  def __sub__(self, other):
    return Power(self.elecPower - other.elecPower, self.fuelPower - other.fuelPower)
  
  def __str__(self):
    val = ""
    for pwr in [self.elecPower, self.fuelPower]:
      if (pwr.value !=0):
        if (val != ""): val += ", "
        val += str(pwr)
    if (val != ""):
      return val
    else:
      return "None"


Definition of heat transfer machines classes, definition forward task

ToDo: definition reverse task, Heat Pump

In [0]:
class EItem:
  def __init__(self):
    self.active = True
    
    self.pwr = None
    self.pwr_factor = 1
    self.htfin = None
    self.htfout = None
    
    self.htfin_min = HTF()
    self.htfin_max = HTF()
    
    self.htfout_min = HTF()
    self.htfout_max = HTF()
    
    self.label = None

In [0]:
class EMachine(EItem):
  # input values: Efficiency efficiency
  def __init__(self, efficiency):
    super().__init__()
    
    self.efficiency = efficiency
    self.genEff = 0
    
    self.pwrin_min = 0
    self.pwrin_max = 0

  # defined values: HTF htfin
  # calculated values: Power pwr, HTF htfout
  def forwardtask(self):
    # power consumption
    self.pwruse = Power(self.pwrin_max * self.pwr_factor)
    # htf input with volume flow limited by htfin_max and htfout_max
    self.htfin.volflow = min(dropNone(self.htfin.volflow, self.htfin_max.volflow, self.htfout_max.volflow))
    
    # htf output is an input...
    self.htfout = HTF() + self.htfin
    # ...plus temperature raise
    self.htfout.temperature += self.pwruse * self.efficiency / (self.htfin.volflow * self.htfin.getDens() * self.htfin.getCp())
    
    # power generation
    self.pwrgen = Power(ElecPower(self.pwruse.fuelPower.value * self.genEff))
    
    # power balance
    self.pwr = self.pwrgen - self.pwruse

  def splitOutput(pwr, efficiency):
    elecPwr = ElecPower(pwr / efficiency.elecEff) if efficiency.elecEff != 0 else 0
    fuelPwr = FuelPower(pwr / efficiency.fuelEff) if efficiency.fuelEff != 0 else 0
    return Power(elecPwr, fuelPwr)
    
  # defined values: HTF htfin, HTF htfout
  # calculated values: Power pwrin, Power pwrout
  def reversetask(self):
    pwr = (self.htfin.volflow * self.htfin.getDens() * self.htfin.getCp()) * (self.htfout.temperature - self.htfin.temperature)
    self.pwrin = HTMachine.splitOutput(pwr, self.efficiency)
    self.pwrout = Power(ElecPower(self.pwrin.fuelPower.value * self.genEff))
  
  def run(self):
    if ((self.pwr_factor == 0) or (self.htfin.volflow == 0)):
      self.active = False
      
    if (self.active):
      print(cind + "Running " + type(self).__name__ + " " + self.label)
      if (not isNone(self.htfin, self.htfout)):
        print(cind + "Attempting reverse task")
        self.reversetask()
      elif (not isNone(self.htfin)):
        print(cind + "Attempting forward task")
        self.forwardtask()
      else:
        raise ValueError("not enough input for machine")
      print(cind + "Input: " + str(self.htfin) + ", output: " + str(self.htfout))
      print(cind + "Energy balance: " + str(self.pwr))
    else:
      print(cind + type(self).__name__ + " " + self.label + " is not active")
      self.pwr = Power()
      self.pwr_factor = 0
      self.htfin = HTF()
      self.htfout = HTF()
       
  
# Power to heat
class P2H_Machine(EMachine):
  # input values: float elecEff
  def __init__(self, elecEff):
    super().__init__(Efficiency(elecEff = elecEff))
    
# Furnace
class F_Machine(EMachine):
  # input values: float fuelEff
  def __init__(self, fuelEff):
    super().__init__(Efficiency(fuelEff = fuelEff))

# Heat pump
class HP_Machine(EMachine):
  pass

# Cogeneration / combined heat and power
class CHP_Machine(EMachine):
  # input values: float fuelEff, float genEff
  def __init__(self, fuelEff, genEff):
    super().__init__(Efficiency(fuelEff = fuelEff))
    self.genEff = genEff
    
# Bypass
class BP_Machine(EMachine):
  def __init__(self):
    super().__init__(Efficiency())
    self.pwrin_max = Power()

Coupling variations

In [0]:
class EConnection(EItem):
  def __init__(self, *elements, connection):
    super().__init__()
    
    self.elements = list()
    self.connection = connection.lower()
    
    for element in elements:
      self.elements.append(element)

  def computeParallel(self):
    self.htfout = HTF()
    self.pwr = Power()
    
    # pwr input for the first element
    htfin_res = HTF() + self.htfin

    for element in self.elements:
      # htf as input 
      element.htfin = HTF() + htfin_res
      element.run()

      # pwr balance
      self.pwr += element.pwr
      # mix htf output
      self.htfout += element.htfout
      
      # subtract used htf
      htfin_res -= element.htfin
      
      print("")

    if (htfin_res.volflow != 0):
      print (cind + "Rejected " + str(htfin_res))
      self.htfin -= htfin_res

  def computeSeries(self):
    self.pwr = Power()

    htfin_element = HTF(self.htfin.temperature, 
                       min(dropNone(self.htfin.volflow, self.htfin_max.volflow, self.htfout_max.volflow)))

    for element in self.elements:
      
      element.htfin = HTF() + htfin_element
      
      element.run()

      self.pwr += element.pwr
      htfin_res = htfin_element - element.htfin
      htfin_element = element.htfout + htfin_res

    self.htfout = htfin_element
  
  def run(self):
    print(cind + "Running " + self.connection + " " + type(self).__name__ + " " + self.label)
    cind.inc()
    if (self.connection[0] == "p"):
      self.computeParallel()
    elif (self.connection[0] == "s"):
      self.computeSeries()
    else:
      raise ValueError("connection unknown")
    cind.dec()
    print(cind + "Input: " + str(self.htfin) + ", output: " + str(self.htfout))
    print(cind + "Energy balance: " + str(self.pwr))
    print("")

forward task

In [13]:
# Example configuration

htfin = HTF(70,0.09)
cind.reset()

p2h_1 = P2H_Machine(1.0)
p2h_1.label = "P2H_1"
p2h_1.pwrin_max = ElecPower(100,"k")
p2h_1.htfout_max = HTF(999, 0.01)

p2h_2 = P2H_Machine(1.0)
p2h_2.label = "P2H_2"
p2h_2.pwrin_max = ElecPower(50,"k")
p2h_2.htfout_max = HTF(999, 0.01)

fur_1 = F_Machine(0.92)
fur_1.label = "FUR_1"
fur_1.pwrin_max = FuelPower(6.52, "M")
fur_1.htfout_max = HTF(999, 0.057)

fur_2 = F_Machine(0.92)
fur_2.label = "FUR_2"
fur_2.pwrin_max = FuelPower(5.44, "M")
fur_2.htfout_max = HTF(999, 0.047)

chp_1 = CHP_Machine(0.489, 0.411)
chp_1.label = "CHP_1"
chp_1.pwrin_max = FuelPower(2, "M")
chp_1.htfout_max = HTF(999, 0.010)

chp_2 = CHP_Machine(0.499, 0.407)
chp_2.label = "CHP_2"
chp_2.pwrin_max = FuelPower(2.42, "M")
chp_2.htfout_max = HTF(999, 0.0103)

bps_1 = BP_Machine()
bps_1.label = "BPS_1"
bps_1.htfout_max = HTF(999, 0.02)

bps_2 = BP_Machine()
bps_2.label = "BPS_2"
bps_2.htfout_max = HTF(999, 0.0097)

stage_1_1 = EConnection(chp_1, fur_1, connection = "parallel")
stage_1_1.label = "STAGE_1_1"
stage_1_2i = EConnection(p2h_1, p2h_2, connection = "parallel")
stage_1_2i.label = "STAGE_1_2I"
stage_1_2 = EConnection(chp_2, fur_2, stage_1_2i, connection = "parallel")
stage_1_2.label = "STAGE_1_2"

system_1 = EConnection(stage_1_1, stage_1_2, connection = "serial")
system_1.label = "SYSTEM_1"
system_1.htfin = htfin
system_1.run()


Running serial EConnection SYSTEM_1
  Running parallel EConnection STAGE_1_1
    Running CHP_Machine CHP_1
    Attempting forward task
    Input: water at 70 °C, 0.01 m3/s, output: water at 93.8826 °C, 0.01 m3/s
    Energy balance: ElecPower 822 kW, FuelPower -2 MW

    Running F_Machine FUR_1
    Attempting forward task
    Input: water at 70 °C, 0.057 m3/s, output: water at 95.6982 °C, 0.057 m3/s
    Energy balance: FuelPower -6.52 MW
    Mixing water at 93.8826 °C, 0.01 m3/s with water at 95.6982 °C, 0.057 m3/s -> water at 95.4272 °C, 0.067 m3/s

    Rejected water at 70 °C, 0.023 m3/s
  Input: water at 70 °C, 0.067 m3/s, output: water at 95.4272 °C, 0.067 m3/s
  Energy balance: ElecPower 822 kW, FuelPower -8.52 MW

  Mixing water at 95.4272 °C, 0.067 m3/s with water at 70 °C, 0.023 m3/s -> water at 88.9292 °C, 0.09 m3/s
  Running parallel EConnection STAGE_1_2
    Running CHP_Machine CHP_2
    Attempting forward task
    Input: water at 88.9292 °C, 0.0103 m3/s, output: water at 117